In [25]:
import base64
import requests
import os
from dotenv import load_dotenv
from mypy_extensions import TypedDict
from typing import Dict
from requests.auth import HTTPBasicAuth
from dateutil import parser

load_dotenv('../.env')

user_key = os.environ.get('METRC_USER_KEY')
vendor_key_CA = os.environ.get('METRC_VENDOR_KEY_CA')

if not user_key or not vendor_key_CA:
    raise Exception('METRC_USER_KEY or METRC_VENDOR_KEY_CA not set')

auth_dict = {'vendor_key': vendor_key_CA, 'user_key': user_key}


In [26]:
licenses = [
    'C11-0000752-LIC',
    'C11-0000995-LIC',
    'CC11-0000578-LIC',
    'CCL19-0001462',
    'CDPH-1000316'
]

class REST(object):
    
    def __init__(self, auth_dict, license_id):
        self.auth = HTTPBasicAuth(auth_dict['vendor_key'], auth_dict['user_key'])
        self.license_id = license_id
        self.base_url = 'https://api-ca.metrc.com'
        
    def get(self, path, time_range=None):
        url = self.base_url + path
        url += '?licenseNumber=' + self.license_id
        
        if time_range:
            lastModifiedStart = parser.parse(time_range[0]).isoformat()
            lastModifiedEnd = parser.parse(time_range[1]).isoformat()
            url += '&lastModifiedStart=' + lastModifiedStart + '&lastModifiedEnd=' + lastModifiedEnd
        
        print(url)
        resp = requests.get(url, auth=self.auth)
        if not resp.ok:
            raise Exception('Code: {}. Reason: {}'.format(resp.status_code, resp.reason))
        
        return resp.content
        
licenses = [licenses[0]]
for i in range(len(licenses)):
    license_id = licenses[i]
    rest = REST(auth_dict=auth_dict, license_id=license_id)
    #resp = rest.get('/facilities/v1')
    #resp = rest.get('/sales/v1/receipts/active')
    #resp = rest.get('/transfers/v1/types')
    resp = rest.get('/transfers/v1/incoming', time_range=('04/12/2021', '04/13/2021'))
    print(resp.content)

https://api-ca.metrc.com/transfers/v1/incoming?licenseNumber=C11-0000752-LIC&lastModifiedStart=2021-04-12T00:00:00&lastModifiedEnd=2021-04-13T00:00:00


Exception: Code: 401. Reason: Unauthorized